In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib widget

from toolz import *

import os
import pickle
import json
import pathlib

In [3]:
lmap = compose(list, map)

In [4]:
def list_of_str_to_list_of_json(lst):
    it = map(lambda x: x.replace('\xa0', ' '), lst)
    it = filter(lambda x: x != '', it)
    it = lmap(json.loads, it)
    return it

In [40]:
data_path = pathlib.Path('data')

In [ ]:
# with open('foodie_favorites_dataset/first_reviews.json', 'r') as f:
#     rewiews_lines = f.readlines() #json.load(f)

# rewiews_first = json.loads(rewiews_lines[1])

# with open('foodie_favorites_dataset/restaurants.json', 'r') as f:
#     for _ in range(1):
#         f.readline()
#     restaurant_nth = json.loads(f.readline())

# for i in range(len(rewiews_first['reviews'])):
#     print(rewiews_first['reviews'][i]['comment'])
#     print('\n' + '-'*100 + '\n')

list_vypechka = []
for file in data_path.joinpath('vypechka-deserty').iterdir():
    list_vypechka.append(file.read_text(encoding='utf-8'))

vypechka_path_to_add =  data_path.joinpath('data_file_vypechka_evropeyskaya.json')
with open(vypechka_path_to_add, 'r', encoding='utf-8') as f:
    list_vypechka += f.readlines()

vypechka_path_to_add =  data_path.joinpath('data_file.json')
with open(vypechka_path_to_add, 'r', encoding='utf-8') as f:
    list_vypechka += f.readlines()

for path in ['data_file_osnovnye_blyda.json',
             'data_file_Завтраки_Напитки_Паста_и_пицца_Соусы_и_маринады_Супы.json',
             'data_file_Закуски_Салаты.json']:
    path_obj = data_path.joinpath(path)
    with open(path_obj, 'r', encoding='utf-8') as f:
        path_list = f.readlines()
    json_form = json.dumps(list_of_str_to_list_of_json(list_vypechka), indent=2, ensure_ascii=False)
    path_obj.write_text(json_form)

json_form = json.dumps(list_of_str_to_list_of_json(list_vypechka), indent=2, ensure_ascii=False)
data_path.joinpath('vypechka-deserty.json').write_text(json_form)


In [41]:
res = {}
for elt in data_path.joinpath('old').iterdir():
    if not elt.is_dir():
        res[str(elt)] = pd.json_normalize(json.loads(elt.read_text(encoding='utf-8')))

In [35]:
res_lst = list(res.values())
res_lst[0]

,title,category,cuisine,menu,description
0,Фаршированные белые грибы с сыром,Закуски,Русская кухня,Еда без ограничений,
1,"Рулеты из огурца, картофельного пюре и сельди",Закуски,Русская кухня,Еда без ограничений,
2,Глазированная репа,Закуски,Русская кухня,Еда без ограничений,
3,Баклажаны с грибами в сметанном соусе,Закуски,Русская кухня,Еда без ограничений,
4,Оладьи из малоглазого макруруса,Закуски,Русская кухня,Еда без ограничений,
...,...,...,...,...,...
7632,Мидии с пармезаном и белым вином,Салаты,Бельгийская кухня,Еда без ограничений,
7633,Салат с подкопченой семгой и хурмой,Салаты,Аргентинская кухня,Еда без ограничений,
7634,"Салат из мяса, овощей и моцареллы с соусом чим...",Салаты,Аргентинская кухня,Еда без ограничений,
7635,Биф-салат,Салаты,Аргентинская кухня,Еда без ограничений,


In [36]:
df_concatenated = pd.concat(res_lst).reset_index(drop=True)
df_concatenated

,title,category,cuisine,menu,description
0,Фаршированные белые грибы с сыром,Закуски,Русская кухня,Еда без ограничений,
1,"Рулеты из огурца, картофельного пюре и сельди",Закуски,Русская кухня,Еда без ограничений,
2,Глазированная репа,Закуски,Русская кухня,Еда без ограничений,
3,Баклажаны с грибами в сметанном соусе,Закуски,Русская кухня,Еда без ограничений,
4,Оладьи из малоглазого макруруса,Закуски,Русская кухня,Еда без ограничений,
...,...,...,...,...,...
40937,Вафли на йогурте,Выпечка и десерты,Бельгийская кухня,Еда без ограничений,
40938,Сырные шарики с виноградом,Выпечка и десерты,Бельгийская кухня,Еда без ограничений,
40939,Печенье альфахорес (alfajores),Выпечка и десерты,Аргентинская кухня,Еда без ограничений,
40940,Аргентинский шоколадный торт,Выпечка и десерты,Аргентинская кухня,Еда без ограничений,


In [37]:
df_with_description = df_concatenated[df_concatenated['description'] != '']#.reset_index(drop=True)
df_with_description

,title,category,cuisine,menu,description
97,Маринованные огурцы сладкие,Закуски,Русская кухня,Еда без ограничений,"Количество ингредиентов рассчитано на 1,5 литр..."
127,Гречневая лапша с яйцами и сыром тофу,Закуски,Японская кухня,Еда без ограничений,"Гречневая лапша соба считается менее вредной, ..."
128,Роллы «Калифорния» с крабом,Закуски,Японская кухня,Еда без ограничений,"«Калифорния», придуманная японцем Итиро Масито..."
132,Онигири с тунцом,Закуски,Японская кухня,Еда без ограничений,Онигири — японские порционные колобки из риса ...
133,Темпура из овощей и морепродуктов,Закуски,Японская кухня,Еда без ограничений,"У слова «темпура» происхождение португальское,..."
...,...,...,...,...,...
40905,Солодовый крем,Выпечка и десерты,Эстонская кухня,Еда без ограничений,Рецепт взят из книги Ники Ганич «География на ...
40907,Слоеные пирожки с грибами,Выпечка и десерты,Эстонская кухня,Еда без ограничений,Рецепт взят из книги Ники Ганич «География на ...
40922,Бельгийские вафли с мороженным и взбитыми слив...,Выпечка и десерты,Бельгийская кухня,Еда без ограничений,"Бельгийскими их называют в России, на родине ж..."
40925,Льежские вафли,Выпечка и десерты,Бельгийская кухня,Еда без ограничений,Еще одна вафельная разновидность родом из Бель...


In [38]:
it = df_concatenated.groupby('cuisine').count()
it.sort_values('title', ascending=False).head(30)

,title,category,menu,description
cuisine,,,,
Европейская кухня,13167,13167,13167,13167
Русская кухня,6633,6633,6633,6633
Итальянская кухня,4422,4422,4422,4422
Французская кухня,2628,2628,2628,2628
Американская кухня,2069,2069,2069,2069
Авторская кухня,1932,1932,1932,1932
Китайская кухня,734,734,734,734
Мексиканская кухня,585,585,585,585
Японская кухня,571,571,571,571
